In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt
import jax
from jax import lax
from deluca.envs import MountainCar
from deluca.agents import ILQR


W1211 06:30:42.848357 4630330816 xla_bridge.py:131] No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [4]:
agent = ILQR()
agent.train(MountainCar(), 10)

iLQR : t = 2, c = 0.050000
iLQR : t = 3, c = 0.002205
iLQR : t = 4, c = 0.000242
iLQR : t = 5, c = 0.000052
iLQR : t = 6, c = 0.000019
iLQR : t = 7, c = 0.000012
iLQR : t = 8, c = 0.000010
iLQR : t = 9, c = 0.000010
iLQR : t = 10, c = 0.000009
iLQR : t = 11, c = 0.000009
iLQR : t = 12, c = 0.000008


In [5]:
def loop(context, x):
    env, agent = context
    control = agent(env.state)
    _, reward, _, _ = env.step(control)
    return (env, agent), reward

In [14]:
 # for loop version
T = 100
env = MountainCar()
print(env.reset())
print('agent.t = ' + str(agent.t))
reward = 0
for i in range(T):
    (env, agent), r = loop((env, agent), 0)
    reward += r
reward_forloop = reward
print('reward_forloop = ' + str(reward_forloop))

[-0.09652638  0.        ]
agent.t = 0
reward_forloop = 1.683697951614514e-05


In [13]:
# scan version
T = 100
env = MountainCar()
print(env.reset())
agent.reset()
xs = jnp.array(jnp.arange(T))
_,reward_scan = lax.scan(loop, (env, agent), xs)
# correctness test
print('reward_scan sum = ' + str(jnp.sum(reward_scan)))

[-0.09652638  0.        ]
reward_scan sum = 1.6836979516145185e-05
